# Objective: Build an Interactive Dashboard To Surface Trends & Patterns in Provider MIPS Scores and Measures

### 1. Read in the data

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import streamlit as st
pd.set_option('display.max_columns', None)


In [21]:
#using 3 files: CMS Physician Compare (provider characteristics) and MIPS measures datasets
#provider characteristics
docs_raw = pd.read_csv('../data/raw/DAC_NationalDownloadableFile.csv', dtype='str', encoding='latin1')
#doctors individual MIPS measures
mm_raw = pd.read_csv('../data/raw/ec_public_reporting.csv', dtype='str')
#doctors overall MIPS scores
mips_raw = pd.read_csv('../data/raw/ec_score_file.csv', dtype='str')

### 2. Clean the data (missing values, duplicates, outliers)

In [22]:
#drop duplicates
docs_raw.drop_duplicates(inplace=True)
mm_raw.drop_duplicates(inplace=True)
mips_raw.drop_duplicates(inplace=True)
#get rid of column names extra spaces
docs_raw.columns = docs_raw.columns.str.strip()
mm_raw.columns = mm_raw.columns.str.strip()
mips_raw.columns = mips_raw.columns.str.strip()

In [23]:
docs_raw.drop(columns=['Ind_PAC_ID', 'Ind_enrl_ID','mid_nm','suff', 'sec_spec_2', 'sec_spec_3', 'sec_spec_4', 'sec_spec_all',
       'org_nm', 'adr_ln_1', 'adr_ln_2','ln_2_sprs', 'cty', 'phn_numbr', 'ind_assgn', 'grp_assgn','adrs_id'],inplace=True)

In [24]:
mm_raw.drop(columns=['Ind_PAC_ID','lst_nm','frst_nm','APM_affl_1', 'APM_affl_2',
       'APM_affl_3','collection_type', 'CCXP_ind'],inplace=True)

In [25]:
mm_raw.columns

Index(['NPI', 'measure_cd', 'measure_title', 'invs_msr', 'attestation_value',
       'prf_rate', 'patient_count', 'star_value', 'five_star_benchmark'],
      dtype='object')

In [26]:
mips_raw.columns
mips_raw.drop(columns=['Provider Last Name', 'Provider First Name',
       'source', 'Facility-based scoring Certification number',
       'Facility Name'],inplace=True)

##### make the master provider level dataframe, joined by NPI

In [27]:
#We don't care about the providers who do not participate in MIPS, for this project. We know 61% don't
#drop nan for NPI keys, there's no point otherwise
docs_raw.dropna(subset=['NPI'], inplace=True)
mm_raw.dropna(subset=['NPI'], inplace=True)
mips_raw.dropna(subset=['NPI'], inplace=True)

In [28]:
#take the overall MIPS scores and left join with MIPS measures (1: many, so more rows)
mips_mm_raw = mips_raw.merge(mm_raw, how='left', on='NPI')

In [29]:
mips_mm_raw

,NPI,Org_PAC_ID,Quality_category_score,PI_category_score,IA_category_score,Cost_category_score,final_MIPS_score_without_CPB,final_MIPS_score,measure_cd,measure_title,invs_msr,attestation_value,prf_rate,patient_count,star_value,five_star_benchmark
0,1003028101,NaN,0,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1003028556,NaN,100,NaN,40,15.19,74.557,78.8092,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1003220351,NaN,NaN,NaN,NaN,NaN,75,75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1003808494,NaN,0,NaN,0,28.192,8.4576,8.4576,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1003813783,NaN,83.6417,NaN,40,NaN,91.8208,94.6966,IA_EC_AHE_1,Engagement of New Medicaid Patients and Follow-up,N,Y,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1196046,1982609657,9931537891,100,100,40,NaN,100,100,PI_EC_PEA_1,Provide Patients Electronic Access to Their He...,N,NaN,95,653,4,100
1196047,1982609657,9931537891,100,100,40,NaN,100,100,PI_EC_PHCDRR_1_EX_1,Immunization Registry Reporting Exclusion,N,Y,NaN,NaN,NaN,NaN
1196048,1982609657,9931537891,100,100,40,NaN,100,100,PI_EC_PHCDRR_4,Public Health Registry Reporting,N,Y,NaN,NaN,NaN,NaN
1196049,1982609657,9931537891,100,100,40,NaN,100,100,PI_EC_PPHI_1,Security Risk Analysis,N,Y,NaN,NaN,NaN,NaN


In [30]:
#then take that and add in the docs information
docs_mips_all_raw = mips_mm_raw.merge(docs_raw, how='left', on='NPI')
docs_mips_all_raw.drop_duplicates(inplace=True)


In [31]:
#handle missing data on master DF
docs_mips_all_raw.isnull().sum().reset_index()

,index,0
0,NPI,0
1,Org_PAC_ID,8004
2,Quality_category_score,367681
3,PI_category_score,804918
4,IA_category_score,220224
5,Cost_category_score,1399257
6,final_MIPS_score_without_CPB,83659
7,final_MIPS_score,0
8,measure_cd,1234711
9,measure_title,1234711


In [32]:
#missing percentage
missing = docs_mips_all_raw.isnull().sum() / len(docs_mips_all_raw) * 100
missing.reset_index(name='% missing')

,index,% missing
0,NPI,0.000000
1,Org_PAC_ID,0.334927
2,Quality_category_score,15.385597
3,PI_category_score,33.681762
4,IA_category_score,9.215265
5,Cost_category_score,58.551855
6,final_MIPS_score_without_CPB,3.500708
7,final_MIPS_score,0.000000
8,measure_cd,51.666434
9,measure_title,51.666434


In [33]:
#droppig org_pac_id where it's null, dropping zip codes that are null
docs_mips_all_raw.dropna(subset=['Org_PAC_ID'], inplace=True)
docs_mips_all_raw.drop(columns=['org_pac_id'], inplace=True)

#### convert all datatypes

In [34]:
#check dtypes
docs_mips_all_raw.dtypes

NPI                             object
Org_PAC_ID                      object
Quality_category_score          object
PI_category_score               object
IA_category_score               object
Cost_category_score             object
final_MIPS_score_without_CPB    object
final_MIPS_score                object
measure_cd                      object
measure_title                   object
invs_msr                        object
attestation_value               object
prf_rate                        object
patient_count                   object
star_value                      object
five_star_benchmark             object
lst_nm                          object
frst_nm                         object
gndr                            object
Cred                            object
Med_sch                         object
Grd_yr                          object
pri_spec                        object
sec_spec_1                      object
Telehlth                        object
num_org_mem              

In [35]:
docs_mips_all_raw['Quality_category_score'] = docs_mips_all_raw['Quality_category_score'].astype('Float64')
docs_mips_all_raw['PI_category_score'] = docs_mips_all_raw['PI_category_score'].astype('Float64')
docs_mips_all_raw['IA_category_score'] = docs_mips_all_raw['IA_category_score'].astype('Float64')
docs_mips_all_raw['Cost_category_score'] = docs_mips_all_raw['Cost_category_score'].astype('Float64')
docs_mips_all_raw['final_MIPS_score_without_CPB'] = docs_mips_all_raw['final_MIPS_score_without_CPB'].astype('Float64')
docs_mips_all_raw['final_MIPS_score'] = docs_mips_all_raw['final_MIPS_score'].astype('Float64')
docs_mips_all_raw['attestation_value'] = docs_mips_all_raw['attestation_value'].apply(lambda x: 'Y' if x == 'Y' else 'N').value_counts()
docs_mips_all_raw['prf_rate'] = docs_mips_all_raw['prf_rate'].astype('Float64')
docs_mips_all_raw['patient_count'] = docs_mips_all_raw['patient_count'] = pd.to_numeric(docs_mips_all_raw['patient_count'], errors='coerce').astype('Int64')
docs_mips_all_raw['star_value'] = docs_mips_all_raw['star_value'] = pd.to_numeric(docs_mips_all_raw['star_value'], errors='coerce').astype('Int64')
docs_mips_all_raw['five_star_benchmark'] = docs_mips_all_raw['five_star_benchmark'] = pd.to_numeric(docs_mips_all_raw['five_star_benchmark'], errors='coerce').astype('Int64')
docs_mips_all_raw['Grd_yr'] = docs_mips_all_raw['Grd_yr'] = pd.to_datetime(docs_mips_all_raw['Grd_yr'], errors='coerce').dt.year
docs_mips_all_raw['num_org_mem'] = docs_mips_all_raw['num_org_mem'] = pd.to_numeric(docs_mips_all_raw['num_org_mem'], errors='coerce').astype('Int64')

In [36]:
df_master = docs_mips_all_raw
df_master.to_parquet('../data/cleaned/df_master.parquet', index=False)

In [37]:
df_master

,NPI,Org_PAC_ID,Quality_category_score,PI_category_score,IA_category_score,Cost_category_score,final_MIPS_score_without_CPB,final_MIPS_score,measure_cd,measure_title,invs_msr,attestation_value,prf_rate,patient_count,star_value,five_star_benchmark,lst_nm,frst_nm,gndr,Cred,Med_sch,Grd_yr,pri_spec,sec_spec_1,Telehlth,num_org_mem,st,zip
8188,1588722581,0042100778,81.358,82.04,40.0,<NA>,85.291,88.566,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,VORALIK,FRANK,M,NaN,UNIVERSITY OF SOUTHERN CALIFORNIA KECK SCHOOL ...,1975.0,DIAGNOSTIC RADIOLOGY,NaN,NaN,<NA>,HI,968144497
8189,1750384285,0042100836,0.0,<NA>,0.0,<NA>,0.0,0.0,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,WEISSINGER,WILLIAM,M,DPM,OTHER,1978.0,PODIATRY,NaN,NaN,<NA>,NY,117433542
8190,1295727808,0042101818,<NA>,83.0,40.0,<NA>,85.55,85.55,IA_EC_CC_2,Implementation of improvements that contribute...,N,NaN,<NA>,<NA>,<NA>,<NA>,WHANG,EUGENE,M,NaN,GEORGE WASHINGTON UNIVERSITY SCHOOL OF MEDICINE,1994.0,DERMATOLOGY,NaN,NaN,6,NY,104652038
8191,1295727808,0042101818,<NA>,83.0,40.0,<NA>,85.55,85.55,IA_EC_PSPA_20,Leadership engagement in regular guidance and ...,N,NaN,<NA>,<NA>,<NA>,<NA>,WHANG,EUGENE,M,NaN,GEORGE WASHINGTON UNIVERSITY SCHOOL OF MEDICINE,1994.0,DERMATOLOGY,NaN,NaN,6,NY,104652038
8192,1295727808,0042101818,<NA>,83.0,40.0,<NA>,85.55,85.55,MIPS_EC_047_overall,Advance Care Plan,N,NaN,3.0,1175,1,100,WHANG,EUGENE,M,NaN,GEORGE WASHINGTON UNIVERSITY SCHOOL OF MEDICINE,1994.0,DERMATOLOGY,NaN,NaN,6,NY,104652038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2606824,1982609657,9931537891,100.0,100.0,40.0,<NA>,100.0,100.0,PI_EC_PEA_1,Provide Patients Electronic Access to Their He...,N,NaN,95.0,653,4,100,MINOR,MARK,M,MD,WEST VIRGINIA UNIVERSITY SCHOOL OF MEDICINE,1982.0,ALLERGY/IMMUNOLOGY,NaN,Y,<NA>,FL,329353145
2606825,1982609657,9931537891,100.0,100.0,40.0,<NA>,100.0,100.0,PI_EC_PHCDRR_1_EX_1,Immunization Registry Reporting Exclusion,N,NaN,<NA>,<NA>,<NA>,<NA>,MINOR,MARK,M,MD,WEST VIRGINIA UNIVERSITY SCHOOL OF MEDICINE,1982.0,ALLERGY/IMMUNOLOGY,NaN,Y,<NA>,FL,329353145
2606826,1982609657,9931537891,100.0,100.0,40.0,<NA>,100.0,100.0,PI_EC_PHCDRR_4,Public Health Registry Reporting,N,NaN,<NA>,<NA>,<NA>,<NA>,MINOR,MARK,M,MD,WEST VIRGINIA UNIVERSITY SCHOOL OF MEDICINE,1982.0,ALLERGY/IMMUNOLOGY,NaN,Y,<NA>,FL,329353145
2606827,1982609657,9931537891,100.0,100.0,40.0,<NA>,100.0,100.0,PI_EC_PPHI_1,Security Risk Analysis,N,NaN,<NA>,<NA>,<NA>,<NA>,MINOR,MARK,M,MD,WEST VIRGINIA UNIVERSITY SCHOOL OF MEDICINE,1982.0,ALLERGY/IMMUNOLOGY,NaN,Y,<NA>,FL,329353145


3. Visualization of trends & patterns By Building Streamlit Interactive Dashboard 

In [ ]:
#High level stats for each relevant column: MIPS scores, each category, gender, specialty

genders = df_master[['NPI','gndr']]
#drop duplicate NPIs
genders = genders.drop_duplicates(subset=['NPI'])
gender_counts = genders.groupby('gndr')['NPI'].count().reset_index()
gender_counts.columns = ['Gender', 'Count']
print(gender_counts)
print(f" females: {gender_counts.iloc[0, 1]}")
print(f" males: {gender_counts.iloc[1, 1]}")

  Gender   Count
0      F  226298
1      M  257884
 females: 226298


In [58]:
genders_counts2 = genders['gndr'].value_counts().reset_index()
genders_counts2.columns = ['gndr', 'count']
genders_counts2


,gndr,count
0,M,257884
1,F,226298
